In [2]:
# Preprocessing des Données - Modèle de Prédiction de Rues Risquées

import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Utilitaires pour le nettoyage géospatial
from math import radians, cos, sin, asin, sqrt
import folium

print("🔧 Début du preprocessing des données")
print("="*60)

# =====================================================================
# 1. CHARGEMENT DES DONNÉES BRUTES
# =====================================================================

print("\n📁 Chargement des datasets...")

def load_csv_file(filename):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        try:
            return pd.read_csv(f'../data/raw/{filename}')
        except FileNotFoundError:
            print(f"⚠️ Fichier non trouvé: {filename}")
            return None
    except Exception as e:
        print(f"❌ Erreur lors du chargement de {filename}: {e}")
        return None

# Chargement des datasets
df_311_raw = load_csv_file('311.csv')
df_crime_raw = load_csv_file('crime-incidents-report.csv')
df_crashes_raw = load_csv_file('vision-zero-crash-records.csv')

# Vérification du chargement
datasets_loaded = {
    '311 Reports': df_311_raw is not None,
    'Crime Incidents': df_crime_raw is not None,
    'Vision Zero Crashes': df_crashes_raw is not None
}

print("\n📊 État du chargement:")
for name, loaded in datasets_loaded.items():
    status = "✅ Chargé" if loaded else "❌ Échec"
    print(f"  {name}: {status}")

# =====================================================================
# 2. NETTOYAGE DU DATASET 311
# =====================================================================

print("\n" + "="*60)
print("🧹 NETTOYAGE DU DATASET 311")
print("="*60)

if df_311_raw is not None:
    df_311_clean = df_311_raw.copy()
    print(f"📋 Dataset initial: {len(df_311_clean):,} lignes")
    
    # ===== Nettoyage des dates =====
    print("\n🕐 Nettoyage des dates...")
    
    # Conversion des colonnes de date
    date_columns = ['open_dt', 'sla_target_dt', 'closed_dt']
    for col in date_columns:
        if col in df_311_clean.columns:
            df_311_clean[col] = pd.to_datetime(df_311_clean[col], errors='coerce')
    
    # Suppression des lignes avec des dates d'ouverture invalides
    before_count = len(df_311_clean)
    df_311_clean = df_311_clean.dropna(subset=['open_dt'])
    after_count = len(df_311_clean)
    print(f"   - Lignes supprimées (dates invalides): {before_count - after_count:,}")
    
    # Ajout de features temporelles dérivées
    df_311_clean['open_year'] = df_311_clean['open_dt'].dt.year
    df_311_clean['open_month'] = df_311_clean['open_dt'].dt.month
    df_311_clean['open_day'] = df_311_clean['open_dt'].dt.day
    df_311_clean['open_weekday'] = df_311_clean['open_dt'].dt.dayofweek
    df_311_clean['open_hour'] = df_311_clean['open_dt'].dt.hour
    
    # ===== Nettoyage des coordonnées géographiques =====
    print("\n🗺️ Nettoyage des coordonnées...")
    
    # Conversion en numérique
    df_311_clean['latitude'] = pd.to_numeric(df_311_clean['latitude'], errors='coerce')
    df_311_clean['longitude'] = pd.to_numeric(df_311_clean['longitude'], errors='coerce')
    
    # Filtrage des coordonnées valides pour Boston
    # Boston bounds: lat [42.1, 42.7], lon [-71.3, -70.8]
    before_geo = len(df_311_clean)
    valid_coords = (
        df_311_clean['latitude'].between(42.1, 42.7) & 
        df_311_clean['longitude'].between(-71.3, -70.8) &
        df_311_clean['latitude'].notna() &
        df_311_clean['longitude'].notna()
    )
    df_311_clean = df_311_clean[valid_coords]
    after_geo = len(df_311_clean)
    print(f"   - Lignes supprimées (coordonnées invalides): {before_geo - after_geo:,}")
    
    # ===== Nettoyage des champs textuels =====
    print("\n📝 Nettoyage des champs textuels...")
    
    # Standardisation des statuts
    status_mapping = {
        'Closed': 'CLOSED',
        'Open': 'OPEN',
        'closed': 'CLOSED',
        'open': 'OPEN'
    }
    if 'case_status' in df_311_clean.columns:
        df_311_clean['case_status'] = df_311_clean['case_status'].map(status_mapping).fillna(df_311_clean['case_status'])
    
    # Nettoyage des noms de rues
    if 'location_street_name' in df_311_clean.columns:
        df_311_clean['location_street_name'] = df_311_clean['location_street_name'].str.upper().str.strip()
    
    # ===== Classification des priorités =====
    print("\n🎯 Classification des priorités...")
    
    # Définition des catégories haute priorité
    high_priority_keywords = [
        'EMERGENCY', 'URGENT', 'SAFETY', 'HAZARD', 'DANGEROUS',
        'POTHOLE', 'TRAFFIC', 'SIGNAL', 'SNOW', 'ICE'
    ]
    
    def classify_priority(title, case_type):
        """Classifie la priorité basée sur le titre et le type"""
        if pd.isna(title):
            return 'LOW'
        
        title_upper = str(title).upper()
        for keyword in high_priority_keywords:
            if keyword in title_upper:
                return 'HIGH'
        
        # Cas spéciaux par type
        if 'POTHOLE' in title_upper or 'TRAFFIC' in title_upper:
            return 'HIGH'
        elif 'GRAFFITI' in title_upper or 'LITTER' in title_upper:
            return 'LOW'
        else:
            return 'MEDIUM'
    
    if 'case_title' in df_311_clean.columns:
        df_311_clean['priority'] = df_311_clean.apply(
            lambda x: classify_priority(x.get('case_title'), x.get('type')), axis=1
        )
    
    print(f"✅ Dataset 311 nettoyé: {len(df_311_clean):,} lignes")
    
    # Statistiques de nettoyage
    if 'priority' in df_311_clean.columns:
        priority_counts = df_311_clean['priority'].value_counts()
        print("   Répartition des priorités:")
        for priority, count in priority_counts.items():
            print(f"     - {priority}: {count:,} ({count/len(df_311_clean)*100:.1f}%)")

# =====================================================================
# 3. NETTOYAGE DU DATASET CRIME
# =====================================================================

print("\n" + "="*60)
print("🚨 NETTOYAGE DU DATASET CRIME")
print("="*60)

if df_crime_raw is not None:
    df_crime_clean = df_crime_raw.copy()
    print(f"📋 Dataset initial: {len(df_crime_clean):,} lignes")
    
    # ===== Nettoyage des dates =====
    print("\n🕐 Nettoyage des dates...")
    
    # Conversion de la date d'incident
    if 'OCCURRED_ON_DATE' in df_crime_clean.columns:
        df_crime_clean['OCCURRED_ON_DATE'] = pd.to_datetime(df_crime_clean['OCCURRED_ON_DATE'], errors='coerce')
        
        # Suppression des lignes avec dates invalides
        before_count = len(df_crime_clean)
        df_crime_clean = df_crime_clean.dropna(subset=['OCCURRED_ON_DATE'])
        after_count = len(df_crime_clean)
        print(f"   - Lignes supprimées (dates invalides): {before_count - after_count:,}")
        
        # Features temporelles dérivées
        df_crime_clean['crime_year'] = df_crime_clean['OCCURRED_ON_DATE'].dt.year
        df_crime_clean['crime_month'] = df_crime_clean['OCCURRED_ON_DATE'].dt.month
        df_crime_clean['crime_day'] = df_crime_clean['OCCURRED_ON_DATE'].dt.day
        df_crime_clean['crime_weekday'] = df_crime_clean['OCCURRED_ON_DATE'].dt.dayofweek
        df_crime_clean['crime_hour'] = df_crime_clean['OCCURRED_ON_DATE'].dt.hour
    
    # ===== Nettoyage des coordonnées =====
    print("\n🗺️ Nettoyage des coordonnées...")
    
    # Conversion et validation
    for col in ['Lat', 'Long']:
        if col in df_crime_clean.columns:
            df_crime_clean[col] = pd.to_numeric(df_crime_clean[col], errors='coerce')
    
    # Filtrage géographique
    before_geo = len(df_crime_clean)
    if 'Lat' in df_crime_clean.columns and 'Long' in df_crime_clean.columns:
        valid_coords = (
            df_crime_clean['Lat'].between(42.1, 42.7) & 
            df_crime_clean['Long'].between(-71.3, -70.8) &
            df_crime_clean['Lat'].notna() &
            df_crime_clean['Long'].notna()
        )
        df_crime_clean = df_crime_clean[valid_coords]
    after_geo = len(df_crime_clean)
    print(f"   - Lignes supprimées (coordonnées invalides): {before_geo - after_geo:,}")
    
    # ===== Classification de la gravité =====
    print("\n⚖️ Classification de la gravité...")
    
    # Définition des crimes par niveau de gravité
    high_severity_crimes = [
        'MURDER', 'HOMICIDE', 'ASSAULT - AGGRAVATED', 'ROBBERY', 
        'RAPE', 'WEAPON', 'SHOOTING'
    ]
    
    medium_severity_crimes = [
        'BURGLARY', 'LARCENY', 'MOTOR VEHICLE ACCIDENT', 'ASSAULT - SIMPLE',
        'VANDALISM', 'THREATS'
    ]
    
    def classify_crime_severity(offense_group, description):
        """Classifie la gravité d'un crime"""
        if pd.isna(offense_group):
            return 'LOW'
        
        offense_upper = str(offense_group).upper()
        
        # Vérification haute gravité
        for crime in high_severity_crimes:
            if crime in offense_upper:
                return 'HIGH'
        
        # Vérification gravité moyenne
        for crime in medium_severity_crimes:
            if crime in offense_upper:
                return 'MEDIUM'
        
        # Cas spéciaux
        if 'INVESTIGATE' in offense_upper or 'SERVICE' in offense_upper:
            return 'LOW'
        
        return 'MEDIUM'
    
    if 'OFFENSE_CODE_GROUP' in df_crime_clean.columns:
        df_crime_clean['severity'] = df_crime_clean.apply(
            lambda x: classify_crime_severity(
                x.get('OFFENSE_CODE_GROUP'), 
                x.get('OFFENSE_DESCRIPTION')
            ), axis=1
        )
        
        # Gestion des shootings
        if 'SHOOTING' in df_crime_clean.columns:
            df_crime_clean.loc[df_crime_clean['SHOOTING'] == 1, 'severity'] = 'HIGH'
    
    print(f"✅ Dataset Crime nettoyé: {len(df_crime_clean):,} lignes")
    
    # Statistiques de gravité
    if 'severity' in df_crime_clean.columns:
        severity_counts = df_crime_clean['severity'].value_counts()
        print("   Répartition par gravité:")
        for severity, count in severity_counts.items():
            print(f"     - {severity}: {count:,} ({count/len(df_crime_clean)*100:.1f}%)")

# =====================================================================
# 4. NETTOYAGE DU DATASET VISION ZERO (CRASHES)
# =====================================================================

print("\n" + "="*60)
print("🚗 NETTOYAGE DU DATASET VISION ZERO")
print("="*60)

if df_crashes_raw is not None:
    df_crashes_clean = df_crashes_raw.copy()
    print(f"📋 Dataset initial: {len(df_crashes_clean):,} lignes")
    
    # ===== Nettoyage des dates =====
    print("\n🕐 Nettoyage des dates...")
    
    if 'dispatch_ts' in df_crashes_clean.columns:
        df_crashes_clean['dispatch_ts'] = pd.to_datetime(df_crashes_clean['dispatch_ts'], errors='coerce')
        
        # Suppression des lignes avec dates invalides
        before_count = len(df_crashes_clean)
        df_crashes_clean = df_crashes_clean.dropna(subset=['dispatch_ts'])
        after_count = len(df_crashes_clean)
        print(f"   - Lignes supprimées (dates invalides): {before_count - after_count:,}")
        
        # Features temporelles
        df_crashes_clean['crash_year'] = df_crashes_clean['dispatch_ts'].dt.year
        df_crashes_clean['crash_month'] = df_crashes_clean['dispatch_ts'].dt.month
        df_crashes_clean['crash_day'] = df_crashes_clean['dispatch_ts'].dt.day
        df_crashes_clean['crash_weekday'] = df_crashes_clean['dispatch_ts'].dt.dayofweek
        df_crashes_clean['crash_hour'] = df_crashes_clean['dispatch_ts'].dt.hour
    
    # ===== Nettoyage des coordonnées =====
    print("\n🗺️ Nettoyage des coordonnées...")
    
    # Conversion des coordonnées
    for col in ['lat', 'long']:
        if col in df_crashes_clean.columns:
            df_crashes_clean[col] = pd.to_numeric(df_crashes_clean[col], errors='coerce')
    
    # Filtrage géographique
    before_geo = len(df_crashes_clean)
    if 'lat' in df_crashes_clean.columns and 'long' in df_crashes_clean.columns:
        valid_coords = (
            df_crashes_clean['lat'].between(42.1, 42.7) & 
            df_crashes_clean['long'].between(-71.3, -70.8) &
            df_crashes_clean['lat'].notna() &
            df_crashes_clean['long'].notna()
        )
        df_crashes_clean = df_crashes_clean[valid_coords]
    after_geo = len(df_crashes_clean)
    print(f"   - Lignes supprimées (coordonnées invalides): {before_geo - after_geo:,}")
    
    # ===== Standardisation des modes de transport =====
    print("\n🚦 Standardisation des modes de transport...")
    
    # Mapping des modes de transport
    mode_mapping = {
        'mv': 'MOTOR_VEHICLE',
        'bike': 'BICYCLE',
        'ped': 'PEDESTRIAN'
    }
    
    if 'mode_type' in df_crashes_clean.columns:
        df_crashes_clean['mode_type_clean'] = df_crashes_clean['mode_type'].map(mode_mapping)
        df_crashes_clean['mode_type_clean'] = df_crashes_clean['mode_type_clean'].fillna('OTHER')
    
    # ===== Classification de la gravité des accidents =====
    print("\n⚖️ Classification de la gravité...")
    
    def classify_crash_severity(mode_type, location_type):
        """Classifie la gravité d'un accident"""
        # Les accidents impliquant piétons et vélos sont plus graves
        if mode_type in ['PEDESTRIAN', 'BICYCLE']:
            if location_type == 'Intersection':
                return 'HIGH'
            else:
                return 'MEDIUM'
        elif mode_type == 'MOTOR_VEHICLE':
            if location_type == 'Intersection':
                return 'MEDIUM'
            else:
                return 'LOW'
        else:
            return 'MEDIUM'
    
    if 'mode_type_clean' in df_crashes_clean.columns:
        df_crashes_clean['crash_severity'] = df_crashes_clean.apply(
            lambda x: classify_crash_severity(
                x.get('mode_type_clean'), 
                x.get('location_type')
            ), axis=1
        )
    
    print(f"✅ Dataset Crashes nettoyé: {len(df_crashes_clean):,} lignes")
    
    # Statistiques des modes de transport
    if 'mode_type_clean' in df_crashes_clean.columns:
        mode_counts = df_crashes_clean['mode_type_clean'].value_counts()
        print("   Répartition par mode de transport:")
        for mode, count in mode_counts.items():
            print(f"     - {mode}: {count:,} ({count/len(df_crashes_clean)*100:.1f}%)")

# =====================================================================
# 5. HARMONISATION DES DATASETS
# =====================================================================

print("\n" + "="*60)
print("🔗 HARMONISATION DES DATASETS")
print("="*60)

# ===== Standardisation des colonnes temporelles =====
print("\n⏰ Standardisation des colonnes temporelles...")

# Créer un format uniforme pour tous les datasets
def standardize_temporal_columns(df, date_col, prefix):
    """Standardise les colonnes temporelles avec un préfixe commun"""
    result_df = df.copy()
    
    # Colonnes standardisées
    result_df[f'{prefix}_datetime'] = df[date_col]
    result_df[f'{prefix}_year'] = df[date_col].dt.year
    result_df[f'{prefix}_month'] = df[date_col].dt.month
    result_df[f'{prefix}_day'] = df[date_col].dt.day
    result_df[f'{prefix}_weekday'] = df[date_col].dt.dayofweek
    result_df[f'{prefix}_hour'] = df[date_col].dt.hour
    result_df[f'{prefix}_date'] = df[date_col].dt.date
    
    return result_df

# Application de la standardisation
standardized_datasets = {}

if 'df_311_clean' in locals():
    standardized_datasets['311'] = standardize_temporal_columns(df_311_clean, 'open_dt', 'incident')
    # Ajout de colonnes identificatrices
    standardized_datasets['311']['data_source'] = '311_REPORTS'
    standardized_datasets['311']['incident_type'] = '311'

if 'df_crime_clean' in locals():
    standardized_datasets['crime'] = standardize_temporal_columns(df_crime_clean, 'OCCURRED_ON_DATE', 'incident')
    standardized_datasets['crime']['data_source'] = 'CRIME_INCIDENTS'
    standardized_datasets['crime']['incident_type'] = 'CRIME'

if 'df_crashes_clean' in locals():
    standardized_datasets['crashes'] = standardize_temporal_columns(df_crashes_clean, 'dispatch_ts', 'incident')
    standardized_datasets['crashes']['data_source'] = 'TRAFFIC_CRASHES'
    standardized_datasets['crashes']['incident_type'] = 'CRASH'

# ===== Standardisation des coordonnées =====
print("\n🗺️ Standardisation des coordonnées...")

def standardize_coordinates(df, lat_col, lon_col):
    """Standardise les colonnes de coordonnées"""
    result_df = df.copy()
    result_df['latitude'] = df[lat_col]
    result_df['longitude'] = df[lon_col]
    return result_df

# Application aux datasets
for name, df in standardized_datasets.items():
    if name == '311':
        # Déjà standardisé
        pass
    elif name == 'crime':
        standardized_datasets[name] = standardize_coordinates(df, 'Lat', 'Long')
    elif name == 'crashes':
        standardized_datasets[name] = standardize_coordinates(df, 'lat', 'long')

print("✅ Coordonnées standardisées pour tous les datasets")

# ===== Création d'un dataset unifié minimal =====
print("\n🔄 Création d'un dataset unifié...")

unified_columns = [
    'data_source', 'incident_type', 'incident_datetime', 
    'latitude', 'longitude', 'incident_year', 'incident_month', 
    'incident_day', 'incident_weekday', 'incident_hour'
]

unified_datasets = []

for name, df in standardized_datasets.items():
    # Sélection des colonnes communes
    available_cols = [col for col in unified_columns if col in df.columns]
    df_subset = df[available_cols].copy()
    
    # Ajout de colonnes spécifiques selon le type
    if name == '311':
        if 'priority' in df.columns:
            df_subset['severity'] = df['priority']
        else:
            df_subset['severity'] = 'MEDIUM'
    elif name == 'crime':
        if 'severity' in df.columns:
            df_subset['severity'] = df['severity']
        else:
            df_subset['severity'] = 'HIGH'
    elif name == 'crashes':
        if 'crash_severity' in df.columns:
            df_subset['severity'] = df['crash_severity']
        else:
            df_subset['severity'] = 'MEDIUM'
    
    unified_datasets.append(df_subset)

# Combinaison des datasets
if unified_datasets:
    df_unified = pd.concat(unified_datasets, ignore_index=True)
    print(f"✅ Dataset unifié créé: {len(df_unified):,} incidents")
    
    # Statistiques du dataset unifié
    print("\n📊 Statistiques du dataset unifié:")
    source_counts = df_unified['data_source'].value_counts()
    for source, count in source_counts.items():
        print(f"  - {source}: {count:,} ({count/len(df_unified)*100:.1f}%)")
    
    severity_counts = df_unified['severity'].value_counts()
    print("\n  Répartition par gravité globale:")
    for severity, count in severity_counts.items():
        print(f"    - {severity}: {count:,} ({count/len(df_unified)*100:.1f}%)")

# =====================================================================
# 6. VALIDATION DE LA QUALITÉ DES DONNÉES
# =====================================================================

print("\n" + "="*60)
print("✅ VALIDATION DE LA QUALITÉ DES DONNÉES")
print("="*60)

def validate_dataset_quality(df, dataset_name):
    """Valide la qualité d'un dataset nettoyé"""
    print(f"\n🔍 Validation pour {dataset_name}:")
    
    # Comptage des valeurs manquantes
    missing_counts = df.isnull().sum()
    critical_columns = ['latitude', 'longitude', 'incident_datetime']
    
    for col in critical_columns:
        if col in df.columns:
            missing_count = missing_counts[col]
            missing_pct = (missing_count / len(df)) * 100
            if missing_pct > 5:
                print(f"  ⚠️ {col}: {missing_count:,} manquantes ({missing_pct:.1f}%)")
            else:
                print(f"  ✅ {col}: {missing_count:,} manquantes ({missing_pct:.1f}%)")
    
    # Validation temporelle
    if 'incident_datetime' in df.columns:
        # Calcul de la période couverte
        try:
            date_range = df['incident_datetime'].max() - df['incident_datetime'].min()
            print(f"  📅 Période couverte: {date_range.days} jours")
        except Exception as e:
            print(f"  ⚠️ Erreur calcul période: {str(e)}")
        
        # Détection d'anomalies temporelles - version robuste
        try:
            # Convertir toutes les dates en naive datetime pour comparaison
            current_time = pd.Timestamp.now()
            
            # Si les colonnes ont une timezone, on la supprime
            if hasattr(df['incident_datetime'].dtype, 'tz') and df['incident_datetime'].dtype.tz is not None:
                incident_dates = df['incident_datetime'].dt.tz_localize(None)
            else:
                incident_dates = df['incident_datetime']
            
            # S'assurer que current_time est aussi naive
            if hasattr(current_time, 'tz') and current_time.tz is not None:
                current_time = current_time.tz_localize(None)
            
            # Comparaison sécurisée
            future_mask = incident_dates > current_time
            future_count = future_mask.sum()
            
            if future_count > 0:
                print(f"  ⚠️ Dates futures détectées: {future_count:,}")
            else:
                print(f"  ✅ Aucune date future détectée")
                
        except Exception as e:
            print(f"  ⚠️ Impossible de vérifier les dates futures: {str(e)}")
    
    # Validation géospatiale
    if 'latitude' in df.columns and 'longitude' in df.columns:
        try:
            coord_bounds = {
                'lat_min': df['latitude'].min(),
                'lat_max': df['latitude'].max(),
                'lon_min': df['longitude'].min(),
                'lon_max': df['longitude'].max()
            }
            
            # Vérification des limites de Boston
            boston_bounds = {
                'lat_min': 42.1, 'lat_max': 42.7,
                'lon_min': -71.3, 'lon_max': -70.8
            }
            
            outside_bounds = (
                (df['latitude'] < boston_bounds['lat_min']) |
                (df['latitude'] > boston_bounds['lat_max']) |
                (df['longitude'] < boston_bounds['lon_min']) |
                (df['longitude'] > boston_bounds['lon_max'])
            ).sum()
            
            if outside_bounds > 0:
                print(f"  ⚠️ Points hors limites Boston: {outside_bounds:,}")
            else:
                print(f"  ✅ Toutes les coordonnées dans les limites de Boston")
                
        except Exception as e:
            print(f"  ⚠️ Erreur validation géospatiale: {str(e)}")
    
    return True

# Validation des datasets nettoyés
if standardized_datasets:
    for name, df in standardized_datasets.items():
        validate_dataset_quality(df, name.upper())

if 'df_unified' in locals():
    validate_dataset_quality(df_unified, "DATASET UNIFIÉ")

# =====================================================================
# 7. SAUVEGARDE DES DONNÉES NETTOYÉES
# =====================================================================

print("\n" + "="*60)
print("💾 SAUVEGARDE DES DONNÉES NETTOYÉES")
print("="*60)

import os

# Création du dossier de sortie
output_dir = 'data/processed'
os.makedirs("../" + output_dir, exist_ok=True)

# Fonction de sauvegarde flexible
def save_dataframe(df, filename_base, output_dir):
    """Sauvegarde un dataframe en essayant plusieurs formats"""
    saved_path = None
    
    # Essayer Parquet d'abord (plus efficace)
    try:
        parquet_path = f"../{output_dir}/{filename_base}.parquet"
        df.to_parquet(parquet_path, index=False)
        saved_path = parquet_path
        print(f"✅ Sauvegardé: {parquet_path} ({len(df):,} lignes) [Parquet]")
    except ImportError:
        # Si Parquet n'est pas disponible, utiliser CSV
        try:
            csv_path = f"../{output_dir}/{filename_base}.csv"
            df.to_csv(csv_path, index=False)
            saved_path = csv_path
            print(f"✅ Sauvegardé: {csv_path} ({len(df):,} lignes) [CSV - Parquet non disponible]")
        except Exception as e:
            print(f"❌ Erreur sauvegarde {filename_base}: {e}")
    except Exception as e:
        print(f"❌ Erreur sauvegarde Parquet {filename_base}: {e}")
        # Fallback vers CSV
        try:
            csv_path = f"../{output_dir}/{filename_base}.csv"
            df.to_csv(csv_path, index=False)
            saved_path = csv_path
            print(f"✅ Sauvegardé: {csv_path} ({len(df):,} lignes) [CSV - Fallback]")
        except Exception as e2:
            print(f"❌ Erreur sauvegarde CSV {filename_base}: {e2}")
    
    return saved_path

# Sauvegarde des datasets individuels
saved_files = []

for name, df in standardized_datasets.items():
    saved_path = save_dataframe(df, f"cleaned_{name}", output_dir)
    if saved_path:
        saved_files.append(saved_path)

# Sauvegarde du dataset unifié
if 'df_unified' in locals():
    saved_path = save_dataframe(df_unified, "unified_incidents", output_dir)
    if saved_path:
        saved_files.append(saved_path)

# Message d'installation de pyarrow si nécessaire
if any('.csv' in path for path in saved_files):
    print("\n💡 CONSEIL:")
    print("   Pour des performances optimales, installez pyarrow:")
    print("   pip install pyarrow")
    print("   Cela permettra l'utilisation du format Parquet (plus rapide et compact)")

# Sauvegarde des métadonnées
def get_temporal_coverage(df):
    """Calcule la couverture temporelle en gérant les timezones"""
    try:
        if 'incident_datetime' not in df.columns:
            return "N/A"
        
        # Normalisation des timezones pour le calcul
        datetime_series = df['incident_datetime'].copy()
        
        # Conversion robuste des timezones
        if pd.api.types.is_datetime64tz_dtype(datetime_series):
            # Si c'est un datetime avec timezone, convertir en naive
            datetime_series = datetime_series.dt.tz_convert(None)
        elif pd.api.types.is_datetime64_dtype(datetime_series):
            # Si c'est déjà un datetime naive, pas de changement
            pass
        else:
            # Conversion vers datetime si ce n'est pas déjà le cas
            datetime_series = pd.to_datetime(datetime_series)
        
        # Assurer que c'est bien naive maintenant
        if pd.api.types.is_datetime64tz_dtype(datetime_series):
            datetime_series = datetime_series.dt.tz_localize(None)
        
        # Calcul sécurisé du min et max
        min_date = datetime_series.min()
        max_date = datetime_series.max()
        
        # Conversion en string pour éviter les problèmes de sérialisation JSON
        return f"{str(min_date)} to {str(max_date)}"
    except Exception as e:
        return f"Error: {str(e)}"

metadata = {
    'preprocessing_date': datetime.now().isoformat(),
    'datasets_processed': list(standardized_datasets.keys()),
    'total_incidents': len(df_unified) if 'df_unified' in locals() else 0,
    'data_quality': {
        'coordinate_coverage': len(df_unified[df_unified[['latitude', 'longitude']].notna().all(axis=1)]) / len(df_unified) * 100 if 'df_unified' in locals() else 0,
        'temporal_coverage': get_temporal_coverage(df_unified) if 'df_unified' in locals() else "N/A"
    },
    'file_formats_used': ['parquet' if any('.parquet' in path for path in saved_files) else 'csv']
}

metadata_filename = f"../{output_dir}/preprocessing_metadata.json"
import json
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"✅ Métadonnées sauvegardées: {metadata_filename}")

# =====================================================================
# 8. RÉSUMÉ DU PREPROCESSING
# =====================================================================

print("\n" + "="*60)
print("📋 RÉSUMÉ DU PREPROCESSING")
print("="*60)

print("\n🎯 DATASETS TRAITÉS:")
if standardized_datasets:
    for name, df in standardized_datasets.items():
        print(f"  • {name.upper()}: {len(df):,} incidents")

if 'df_unified' in locals():
    print(f"\n📊 DATASET UNIFIÉ: {len(df_unified):,} incidents totaux")
    
    # Calcul sécurisé de la période avec gestion de timezone
    try:
        # Normalisation des timezones pour l'affichage
        datetime_series = df_unified['incident_datetime']
        if hasattr(datetime_series.dtype, 'tz') and datetime_series.dtype.tz is not None:
            datetime_series = datetime_series.dt.tz_localize(None)
        
        min_date = datetime_series.min()
        max_date = datetime_series.max()
        print(f"  • Période: {min_date} à {max_date}")
    except Exception as e:
        print(f"  • Période: Erreur calcul dates - {str(e)}")
    
    print(f"  • Couverture géographique: {len(df_unified[df_unified[['latitude', 'longitude']].notna().all(axis=1)])} incidents avec coordonnées")

print(f"\n💾 FICHIERS SAUVEGARDÉS:")
for filename in saved_files:
    print(f"  • {filename}")

print(f"\n✅ QUALITÉ DES DONNÉES:")
print(f"  • Coordonnées valides: {metadata['data_quality']['coordinate_coverage']:.1f}%")
print(f"  • Période temporelle: {metadata['data_quality']['temporal_coverage']}")

print("\n🚀 PROCHAINE ÉTAPE:")
print("  Les données sont prêtes pour le feature engineering!")
print("  Prochain notebook: 03_feature_engineering.ipynb")

print("\n" + "="*60)
print("✨ PREPROCESSING TERMINÉ AVEC SUCCÈS")
print("="*60)

🔧 Début du preprocessing des données

📁 Chargement des datasets...

📊 État du chargement:
  311 Reports: ✅ Chargé
  Crime Incidents: ✅ Chargé
  Vision Zero Crashes: ✅ Chargé

🧹 NETTOYAGE DU DATASET 311
📋 Dataset initial: 709 lignes

🕐 Nettoyage des dates...
   - Lignes supprimées (dates invalides): 0

🗺️ Nettoyage des coordonnées...
   - Lignes supprimées (coordonnées invalides): 2

📝 Nettoyage des champs textuels...

🎯 Classification des priorités...
✅ Dataset 311 nettoyé: 707 lignes
   Répartition des priorités:
     - MEDIUM: 707 (100.0%)

🚨 NETTOYAGE DU DATASET CRIME
📋 Dataset initial: 185,534 lignes

🕐 Nettoyage des dates...
   - Lignes supprimées (dates invalides): 0

🗺️ Nettoyage des coordonnées...
   - Lignes supprimées (coordonnées invalides): 11,468

⚖️ Classification de la gravité...
✅ Dataset Crime nettoyé: 174,066 lignes
   Répartition par gravité:
     - LOW: 172,803 (99.3%)
     - HIGH: 1,263 (0.7%)

🚗 NETTOYAGE DU DATASET VISION ZERO
📋 Dataset initial: 37,896 lignes

🕐 